In [ ]:
import csv
import os
import pandas as pd
import sqlite3

Setting variables

In [ ]:
path_wd   = os.getcwd() 
path_root = path_wd.rsplit("\\",1)[0]

path_data = path_root + '\\data\\'
path_etl  = path_root + '\\etl\\'

path_user = os.path.expanduser('~')